In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from support import load_dataset
from torch.utils.data import DataLoader
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# -----------------------------
#        INPUT PARAMETERS
# -----------------------------
# Random seed
SEED = 18

# Seach method
EVALUATION_FOR = "EVALUATION_BASELINE"  # choose "EVALUATION_BASELINE" or "EVALUATION_RANDOM" or "EVALUATION_TPE"    

# Final model training settings
NUM_EPOCHS_FINAL = 30

# Fixed hyperparameters (choose your values here)
LR = 1e-3
DROPOUT = 0.5
OPTIMIZER_NAME = "Adam"        # choose "Adam" or "SGD"
BATCH_SIZE = 32
NUM_FILTERS = 32               # number of filters in the first conv layer

# Filenames for saving the final learning curves
PLOTS_DIR = f"plots/{EVALUATION_FOR}"
LEARNING_CURVE_FILENAME = "learning_curve.png"
LOSS_CURVE_FILENAME     = "loss_curve.png"
os.makedirs(PLOTS_DIR, exist_ok=True)

# Other constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
#        END PARAMETERS
# -----------------------------

# Set seed for reproducibility
def set_seed(seed: int = SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

set_seed()

print(f"Device in use: {DEVICE}")
print("Loading datasets…")
train_dataset, test_dataset = load_dataset()
print(f"Train set size: {len(train_dataset)}, Test set size: {len(test_dataset)}")

# Define simple CNN
class CNN(nn.Module):
    def __init__(self, num_filters: int = NUM_FILTERS, dropout: float = DROPOUT):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, num_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters, num_filters * 2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # Dynamically compute flattened size (input assumed 60 × 30)
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 60, 30)
            out = self.features(dummy)
            flat_size = out.view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Training / evaluation helpers
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc  = correct / len(loader.dataset)
    return epoch_loss, epoch_acc

def evaluate(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
    return correct / len(loader.dataset)

def evaluate_loss(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    count = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            count += inputs.size(0)
    return total_loss / count

# Train the model for multiple epochs and record metrics
def train_full_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs: int):
    train_accs = []
    train_losses = []
    test_accs = []
    test_losses = []

    for epoch in range(1, num_epochs + 1):
        # Train on training set
        model.train()
        running_loss = 0.0
        correct = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
        train_loss = running_loss / len(train_loader.dataset)
        train_acc  = correct / len(train_loader.dataset)

        # Evaluate on test set
        model.eval()
        total_loss = 0.0
        correct = 0
        count = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item() * inputs.size(0)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                count += inputs.size(0)
        test_loss = total_loss / count
        test_acc  = correct / count

        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)

        print(
            f"Epoch {epoch:02d}/{num_epochs} | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
            f"Test  Loss: {test_loss:.4f}, Test  Acc: {test_acc:.4f}"
        )

    return train_losses, train_accs, test_losses, test_accs

# Plot learning curves (accuracy + loss) and save under plots/…
def plot_learning_curves(
    epochs,
    train_accs,
    test_accs,
    train_losses,
    test_losses,
    acc_title: str,
    loss_title: str,
    acc_filepath: str,
    loss_filepath: str
):
    # Accuracy plot
    plt.figure(figsize=(8, 5))
    plt.plot(epochs, train_accs, marker='o', label="Train Accuracy")
    plt.plot(epochs, test_accs, marker='s', label="Test Accuracy")
    if train_accs:
        best_train_idx = int(np.argmax(train_accs))
        best_train_val = max(train_accs)
        plt.scatter(epochs[best_train_idx], best_train_val, color='blue')
        plt.text(epochs[best_train_idx], best_train_val + 0.01,
                 f"Max Train Acc: {best_train_val:.2f}", color='blue')
    if test_accs:
        best_test_idx = int(np.argmax(test_accs))
        best_test_val = max(test_accs)
        plt.scatter(epochs[best_test_idx], best_test_val, color='orange')
        plt.text(epochs[best_test_idx], best_test_val + 0.01,
                 f"Max Test Acc: {best_test_val:.2f}", color='orange')
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(acc_title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(acc_filepath, dpi=300)
    plt.show()

    # Loss plot
    plt.figure(figsize=(8, 5))
    plt.plot(epochs, train_losses, marker='o', label="Train Loss")
    plt.plot(epochs, test_losses, marker='s', label="Test Loss")
    if train_losses:
        best_train_loss_idx = int(np.argmin(train_losses))
        best_train_loss_val = min(train_losses)
        plt.scatter(epochs[best_train_loss_idx], best_train_loss_val, color='blue')
        plt.text(epochs[best_train_loss_idx], best_train_loss_val + 0.01,
                 f"Min Train Loss: {best_train_loss_val:.2f}", color='blue')
    if test_losses:
        best_test_loss_idx = int(np.argmin(test_losses))
        best_test_loss_val = min(test_losses)
        plt.scatter(epochs[best_test_loss_idx], best_test_loss_val, color='orange')
        plt.text(epochs[best_test_loss_idx], best_test_loss_val + 0.01,
                 f"Min Test Loss: {best_test_loss_val:.2f}", color='orange')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(loss_title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(loss_filepath, dpi=300)
    plt.show()

# -----------------------------
#      FINAL TRAINING FLOW
# -----------------------------

# 1) Build model and optimizer with fixed hyperparameters
model = CNN(num_filters=NUM_FILTERS, dropout=DROPOUT).to(DEVICE)
optimizer = (
    optim.Adam(model.parameters(), lr=LR)
    if OPTIMIZER_NAME == "Adam"
    else optim.SGD(model.parameters(), lr=LR)
)
criterion = nn.CrossEntropyLoss()

# 2) Prepare DataLoaders for full train and test sets
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# 3) Train the model and record metrics
print("Training the model on the full training set…")
train_losses, train_accs, test_losses, test_accs = train_full_model(
    model,
    train_loader,
    test_loader,
    criterion,
    optimizer,
    DEVICE,
    NUM_EPOCHS_FINAL
)

# 4) Plot and save the final learning & loss curves
epochs = np.arange(1, NUM_EPOCHS_FINAL + 1)
print("Plotting final learning curves…")
plot_learning_curves(
    epochs,
    train_accs,
    test_accs,
    train_losses,
    test_losses,
    acc_title="Baseline Train & Test Accuracy per Epoch",
    loss_title="Baseline Train & Test Loss per Epoch",
    acc_filepath=os.path.join(PLOTS_DIR, LEARNING_CURVE_FILENAME),
    loss_filepath=os.path.join(PLOTS_DIR, LOSS_CURVE_FILENAME)
)


In [ ]:
# -----------------------------
#      SAVE MODEL WEIGHTS
# -----------------------------
# Create a directory for saved models (if it doesn’t exist)
WEIGHTS_DIR = "saved_models"
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# Define a filename (you can include hyperparams or date/time if you like)
WEIGHTS_FILENAME = "cnn_baseline_weights.pth"
weights_path = os.path.join(WEIGHTS_DIR, WEIGHTS_FILENAME)

# Save only the state_dict (recommended):
torch.save(model.state_dict(), weights_path)
print(f"Model weights saved to {weights_path}")